PART 2

In [15]:
from collections import  defaultdict

with open("Data/ES/train") as f:
    es = f.read().splitlines()

with open("Data/RU/train") as f:
    ru = f.read().splitlines()




In [16]:

def estimate_emission_params(train_data,k=1):

    word_sentiment_counts = defaultdict(lambda: defaultdict(int))
    sentiment_counts =  defaultdict(int)
    emission_params = {}
    
    #getting the count(y) and count(y --> x)
    for sentence in train_data:
        try:
            if(sentence!=""):
                x, label = sentence.split(" ")
        except:
            continue

        sentiment_counts[label]+=1

        word_sentiment_counts[label][x]+=1



    #calculating the emission parameters

    for key in word_sentiment_counts:
        for word in word_sentiment_counts[key]:
            emission_params[(word,key)] = word_sentiment_counts[key][word]/(sentiment_counts[key])
    return emission_params

In [17]:
#Getting the data from ES 
# train_es = read_data('./Data/ES/train')

def estimate_emission_params_unk(train_data,k=1):

    word_sentiment_counts = defaultdict(lambda: defaultdict(int))
    sentiment_counts = defaultdict(lambda: 1)
    emission_params = {}
    
    #getting the count(y) and count(y --> x)
    for sentence in train_data:
        try:
            if(sentence!=""):
                x, label = sentence.split(" ")
        except:
            continue

        sentiment_counts[label]+=1

        word_sentiment_counts[label][x]+=1


    #changing count(y->x) when x="#UNK"
    for key in word_sentiment_counts:
        word_sentiment_counts[key]["#UNK#"]=k


    #calculating the emission parameters

    for key in word_sentiment_counts:
        for word in word_sentiment_counts[key]:
            emission_params[(word,key)] = word_sentiment_counts[key][word]/(sentiment_counts[key]+k)
    return emission_params


In [18]:
# results of training using es and ru datasets for part 1
result_es = estimate_emission_params(es)
result_ru = estimate_emission_params(ru)
# results of training using es and ru datasets for part 2
result_es_unk = estimate_emission_params_unk(es)
result_ru_unk = estimate_emission_params_unk(ru)

PART 3

In [19]:
with open("Data/ES/dev.in") as f:
    dev_in_es = f.read().splitlines()
with open("Data/ES/dev.out") as f:
    dev_out_es = f.read().splitlines()
with open("Data/RU/dev.in") as f:
    dev_in_ru = f.read().splitlines()
with open("Data/RU/dev.out") as f:
    test_out_ru = f.read().splitlines()

In [20]:
#FINDING THE MAXIMUM VLLAUE OF e(x|y)


##########################################################
#updtaing the k value for the new strings not seen by the training data
k=0
for word in dev_in_es:
    if word not in result_es_unk:
        k += 1
##########################################################
#Stroing the probablities and tags into holders
em_prm=estimate_emission_params_unk(es,k)
holder={}
holder2={}
for i in em_prm:
    if i[0] not in holder:
        holder[i[0]]=[]
        holder2[i[0]]=[]
    holder[i[0]].append(em_prm[i])   
    holder2[i[0]].append(i[1])   
##########################################################
# Getting the maximum value of the probabiliites and using 
# it to get the index to gewt the correct tag
y_star={}
for i in holder:
    index=holder[i].index(max(holder[i]))
    y_star[i]=holder2[i][index]
##########################################################
# writing y_star into dev.p1.out

with open('dev.p1.out', 'w', encoding='utf-8') as dev_out_file:
    for predicted_labels in y_star:
        dev_out_file.write(f"{predicted_labels}\t{y_star[predicted_labels]}\n")
    dev_out_file.write('\n')

In [21]:
# Calculating the Precision, Recall and F score for the given data
def scores(labels, correct_labels):
    correct = 0
    total_predicted = 0
    for word in labels:
        if labels[word] == correct_labels[word]:
            correct += 1
        total_predicted += 1
        
    precision = correct / total


def F_score(recall,precision):
    v1=1/precision
    v2=1/recall
    f=2/(v1+v2)